In [1]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.05G/1.06G [00:09<00:00, 119MB/s]
100% 1.06G/1.06G [00:09<00:00, 120MB/s]


In [3]:
import zipfile
import os

# Unzip the dataset
with zipfile.ZipFile("dogs-vs-cats.zip", 'r') as zip_ref:
    zip_ref.extractall("dogs-vs-cats")

# Verify the extraction
print("Extracted files:", os.listdir("dogs-vs-cats"))

Extracted files: ['test', 'dogs_vs_cats', 'train']


In [4]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16

In [5]:
conv_base = VGG16(
    weights="imagenet",
    include_top=False,   # here false make the the last dense layer is excluded and only convulational part is taken
    input_shape=(256, 256, 3)
    )

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [7]:
conv_base.trainable = False # here it is done for transfer learning

In [8]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/dogs-vs-cats/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,                                         # for making the dataset usable (data generator)
    image_size = (256, 256)
)
validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/dogs-vs-cats/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256, 256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [9]:
#normalize
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(train_ds , epochs = 10 , validation_data = validation_ds)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 157s 218ms/step - accuracy: 0.8427 - loss: 0.5795 - val_accuracy: 0.8580 - val_loss: 0.3500
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 229ms/step - accuracy: 0.9462 - loss: 0.1339 - val_accuracy: 0.9390 - val_loss: 0.1468
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 129s 206ms/step - accuracy: 0.9632 - loss: 0.0959 - val_accuracy: 0.9340 - val_loss: 0.1739
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 128s 205ms/step - accuracy: 0.9714 - loss: 0.0725 - val_accuracy: 0.9296 - val_loss: 0.1801
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 228ms/step - accuracy: 0.9837 - loss: 0.0453 - val_accuracy: 0.9294 - val_loss: 0.2008
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 128s 206ms/step - accuracy: 0.9788 - loss: 0.0535 - val_accuracy: 0.9312 - val_loss: 0.2264
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 157s 230ms/step - accuracy: 0.9897 - loss: 0.0286 - val_accuracy: 0.9308 - val_loss: 0.2281
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 129s 206ms/step - accuracy: 0.9922 -